Wayne Nixalo - 19-Jan-2018 15:29

Working on how exactly I'll read in data from numbered CSV files, and concatenate them into a single main CSV file.

In [1]:
import pandas as pd
import os

In [9]:
path = 'data/'
csv_name = 'interstage_labels'

In [24]:
csvs = [fname for fname in os.listdir(path) if csv_name in fname and '.csv' in fname]

In [25]:
csvs

['interstage_labels-001505-001505.csv',
 'interstage_labels-000006-000006.csv',
 'interstage_labels-000004-000005.csv',
 'interstage_labels-000002-000003.csv',
 'interstage_labels-000000-000001.csv']

In [34]:
# Initialize DataFrame as first CSV
out_df = pd.read_csv(path + csvs[0])
# append all other CSVs to DataFrame
for i in range(1, len(csvs)):
    out_df = out_df.append(pd.read_csv(path + csvs[i]))

# save new CSV file
# out_df.to_csv(path + csv_name + '.csv', index=False)

In [35]:
out_df

,id,x1,y1,x2,y2
0,001435-001882/001505.jpg,210,102,314,379
0,000000-000412/000006.jpg,210,102,314,379
0,000000-000412/000004.jpg,65,90,209,389
1,000000-000412/000005.jpg,47,69,209,381
0,000000-000412/000002.jpg,58,89,208,390
1,000000-000412/000003.jpg,66,98,214,388
0,000000-000412/000000.jpg,83,72,191,380
1,000000-000412/000001.jpg,52,89,204,381


In [39]:
out_df['id']

1    000000-000412/000005.jpg
1    000000-000412/000003.jpg
1    000000-000412/000001.jpg
Name: id, dtype: object

In [53]:
out_df['id']

array(['001435-001882/001505.jpg', '000000-000412/000006.jpg',
       '000000-000412/000004.jpg', '000000-000412/000005.jpg',
       '000000-000412/000002.jpg', '000000-000412/000003.jpg',
       '000000-000412/000000.jpg', '000000-000412/000001.jpg'],
      dtype=object)

In [63]:
name = '000000-000412/000005.jpg'
[row for row in out_df['id'] if row == name]

['000000-000412/000005.jpg']

In [62]:
[row for row in out_df['id']]

['001435-001882/001505.jpg',
 '000000-000412/000006.jpg',
 '000000-000412/000004.jpg',
 '000000-000412/000005.jpg',
 '000000-000412/000002.jpg',
 '000000-000412/000003.jpg',
 '000000-000412/000000.jpg',
 '000000-000412/000001.jpg']

In [68]:
[(i,row, row==name) for i,row in enumerate(out_df['id'])]

[(0, '001435-001882/001505.jpg', False),
 (1, '000000-000412/000006.jpg', False),
 (2, '000000-000412/000004.jpg', False),
 (3, '000000-000412/000005.jpg', True),
 (4, '000000-000412/000002.jpg', False),
 (5, '000000-000412/000003.jpg', False),
 (6, '000000-000412/000000.jpg', False),
 (7, '000000-000412/000001.jpg', False)]

In [70]:
out_df['id'].iloc[3]

'000000-000412/000005.jpg'

Okay, I'm in a car right now with no internet, so I had to figure out how to find a particular row in a DataFrame. I can use this to extract the G-LOC label from the input dataset; because I forgot to do that in the `cropper.py` program.

In [71]:
import numpy as np

In [75]:
np.where(out_df['id']==name)

(array([3]),)

In [76]:
np.where(out_df['id']==name)[0][0]

3

In [78]:
out_df.iloc[3]['id']

'000000-000412/000005.jpg'

---

Heh, I did this next part waiting for a lecture (6.S094) to start at MIT and I feel really special. Lol...

In [87]:
interstage_ids = ['006861-007364/007306.jpg','006861-007364/007307.jpg','006861-007364/007308.jpg']

In [88]:
interstage_ids = [fpath.split('.jpg')[0] for fpath in interstage_ids]

In [89]:
interstage_ids

['006861-007364/007306', '006861-007364/007307', '006861-007364/007308']

In [85]:
labels_csv = pd.read_csv('data/labels.csv')

In [90]:
label_idxs = [np.where(labels_csv['id'] == fpath) for fpath in interstage_ids]

In [93]:
label_idxs = [item[0][0] for item in label_idxs]

In [94]:
label_idxs

[7306, 7307, 7308]

Back in the car on the way home.

---

I was having an issue with the GLOC classification being recorded as floats. They're suppsoed to be integers.

In [96]:
type(label_idxs[0])

numpy.int64

In [97]:
interstage_labels = [labels_csv.iloc[row]['gloc'] for row in label_idxs]

In [100]:
interstage_labels[0].dtype

dtype('int64')

Aha..

In [103]:
label_idxs[0]

7306

Hmm, maybe Pandas was automtaically converting my ints to floats because of the presence of NaNs? I was concatenating dataframes that didn't have any GLOC column, and while searching for how to save integer arrays I came across a lot of people having trouble storing both integers and NaNs in the same column. It may be a related issue.

I can test exhaustively to confirm (not going to), I could edit the code to cast the GLOC classifications into integers (would'nt change anything becuase it doesnt address the cause issue), or I could leave it be, because I'm going to redo the interstage dataset to include the GLOC column anyway.

Also I can write in a quick flag to see if the clsftns are going in as floats or ints. I expect to get int, meaning they're being changed after entering the dataframe.

Oh wait, unnec. I can just read in the last CSV which was done including GLOC clsfns.

In [105]:
new_df = pd.read_csv(path + 'interstage_labels-001506-001506.csv')

In [106]:
new_df['gloc']

0    0
Name: gloc, dtype: int64

There you have it. Most likely caused by the presence of NaNs. This won't occur in normal operation, and if it does: I'll fix it.

-- Wayne Nixalo -- 23:09

Ah, a bigger issue is that the GLOC column was placed as the first column even though I specified an index of `1` in `cropper.py`. `cropper.py` saves it to the right position. Something got screwed up in `utils.common.csv_stitcher`. Again because of the NaNs?

I have some other strange issues happening when I try to concatenate dataframes from CSVs. I'm go just redo the mini-test dataset when I get home.

Such as this:

```
>>> path = 'data/'
>>> csv_name = 'interstage_labels'
>>> csvs = [fname for fname in os.listdir(path) if csv_name in fname and '.csv' in fname]
>>> csvs.sort()
>>> csvs
['interstage_labels-000000-000001.csv', 'interstage_labels-000002-000003.csv', 'interstage_labels-000004-000005.csv', 'interstage_labels-000006-000006.csv', 'interstage_labels-001505-001505.csv', 'interstage_labels-001506-001506.csv', 'interstage_labels.csv']
```
Issue Here:
```
>>> out_df = pd.read_csv(path + csvs[0])
>>> out_df.head()
                         id  x1  y1   x2   y2
0  000000-000412/000000.jpg  83  72  191  380
1  000000-000412/000001.jpg  52  89  204  381


>>> for i in range(1, len(csvs)-2):
...     out_df = out_df.append(pd.read_csv(path + csvs[i]))
...
>>> out_df.head()
                        1id                        id  x1   x2  y1   y2
0                       NaN  000000-000412/000000.jpg  83  191  72  380
1                       NaN  000000-000412/000001.jpg  52  204  89  381
0  000000-000412/000002.jpg                       NaN  58  208  89  390
1  000000-000412/000003.jpg                       NaN  66  214  98  388
0                       NaN  000000-000412/000004.jpg  65  209  90  389
```